# Introduction to matplotlib

`matplotlib` is the Python plotting package to rule them all. Not because it's the best. Or the easiest to use. Or the fastest. Or... wait, why is it the number 1 plotting package? Nobody knows! But it's everywhere, and making basic plots is... fine. It's really fine.

This is the `pyplot` interface to `matplotlib`. It's the easiest one to use, but it's less flexible than the so-called 'object-oriented' interface.

Here's the official tutorial for the `pyplot` interface: https://matplotlib.org/tutorials/introductory/pyplot.html

Producing the same plot with the object-oriented interface looks like this:

For the rest of this lesson, we're going to use the object-oriented approach.

## Load some data

In [ ]:
import pandas as pd

url = "https://github.com/scienxlab/datasets/raw/refs/heads/main/kgs/panoma_data.xlsx"

df = pd.read_excel(url, sheet_name='data')
df.head()

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">

<h3>EXERCISE</h3>

- Make the following variables, using only the SHRIMPLIN well. For each one, add `.to_numpy()` at the end so we end up with a NumPy array, not a Pandas object.
  - `gr`: the GR log.
  - `phi`: the PHIND log.
  - `depth`: the depth column.
  - `facies`: the facies column.
- Make the following plots:
  - Plot GR on its own. What does the _x_ axis represent?
  - Plot GR vs depth (ie with depth on the _x_ axis).
  - Plot depth vs GR.
  - Add `color='red'` to your call to `ax.plot()`.
  - What happens if you add another line with `ax.set_ylim(840, 920)`?
  - Switch the values in `ax.set_ylim()`.
  - Try instantiating the figure with `plt.subplots(figsize=(2,10))` at the start.
</div>

We can add another subplot:

In [ ]:
# Build this up.
fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(4, 6), sharey=True)

ax0.plot(gr, depth, lw=0.5)
ax0.set_ylim(depth[-1]+10, depth[0]-10)
ax0.set_xlabel('GR [API]')
ax0.set_ylabel('Depth [m]')
ax0.set_title('GR log')
ax0.grid('k', alpha=0.3)

ax1.plot(phi, depth, 'g', lw=0.5)
ax1.set_ylim(depth[-1]+10, depth[0]-10)
ax1.set_xlabel('Phi [v/v]')
ax1.set_title('Phi log')
ax1.grid('k', alpha=0.3)

plt.show()  # <--- Note that you need this in a script.

## `plt.scatter()`

It's also easy to make scatter plots:

We can adjust how the points plot to make it more interesting:

In [ ]:
fig, ax = plt.subplots()

ax.scatter(gr, phi, c=facies, s=10, alpha=0.5, cmap='tab10')
ax.grid(c='k', alpha=0.1)

## `plt.hist()`

We often want to look at the distribution of our data.

Note that we can also pass Pandas objects, eg Series, to `matplotlib`:

In general, `seaborn` makes nicer histograms, and adds a KDE (kernel density estimation) plot. It also prefers data without NaNs though...

In [ ]:
import seaborn as sns

ax = sns.histplot(gr, kde=True, lw=0)

We can also pass DataFrames to Seaborn, with a lot of extra options, eg using columns directly for colour etc.

In [ ]:
ax = sns.kdeplot(df, x='GR', hue='Well Name')

## `plt.imshow()` for raster data

For image-like data, such as slices of seismic, we need a different kind of visualization. 

NB There's also `plt.pcolor` but it's very slow. Use `plt.pcolormesh` instead.

Let's load some seismic data!

In [ ]:
import pooch

spot = pooch.create(path='../data',
                    base_url="https://github.com/scienxlab/datasets/raw/refs/heads/main/nlog",
                    registry={
                        "F3_3x3_16-bit_int.npy": 'md5:b929b851df1bf729e08ebf8f8e2b53d1',
                        "F3_FS4_horizon.npy": 'md5:06efe43cbabf266743e07b53f5af0b86',
                        "F3_FS8_horizon.npy": 'md5:657858149000397c638ce6cf0cd9694d',
                        "F3_Demo_0_FS4.dat": 'md5:68851622ed91d6890ddc500739a3b96d',
                    })

fname = spot.fetch('F3_3x3_16-bit_int.npy')

vol = np.load(fname)
vol

Adding a colorbar requires a little extra code. We can also make it larger.

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">

<h3>Exercise</h3>

- Try plotting a vertical section through the data. You'll need to think about indexing into `vol`.
- Can you make a histogram of the amplitudes? Tip: Use only one slice of the data and use the `ravel()` method on it to change it into a 1D array. If there are NaNs in the data, you may need to deal with them.
</div>

## More `imshow` options

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(vol[:50, :50, 200], interpolation='bicubic')
plt.colorbar(im)

Note that the colorbar is no longer symmetric. We can set the min and max of the display:

In [ ]:
vol.min(), vol.max()

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(vol[:50, :50, 200], vmin=-20_000, vmax=20_000)
plt.colorbar(im)

Best is to use percentiles, eg:

In [ ]:
M = np.percentile(vol, 99.0)

We can choose new colourmaps easily, and post the colorbar.

In [ ]:
fig, ax = plt.subplots()
ax.imshow(vol[..., 300], cmap='gray', vmin=-M, vmax=M)

Note too that matplotlib colourmaps all have reversed versions, just add `_r` to the end of the name.

In [ ]:
fig, ax = plt.subplots()
ax.imshow(vol[..., 300], cmap='RdBu_r', vmin=-M, vmax=M)

We can give the image inline-crossline coordinates or some other survey coordinates:

In [ ]:
fig, ax = plt.subplots()
ax.imshow(vol[:50, :50, 200], extent=[10000, 11000, 200000, 201000], vmin=-M, vmax=M)

Notice that `plt.imshow()` assumes your pixels are square.

## Contour maps and plotting multiple datasets

Let's load a horizon. We have a DAT file, exported from OpendTect. We've left it in its original format, which looks like this:

```
# 1: X
# 2: Y
# 3: Inline
# 4: Crossline
# 5: Z
# - - - - - - - - - -
612076.10	6073980.89	110	550	1069.591283798217773
612126.08	6073982.28	110	552	1069.051265716552734
612176.06	6073983.68	110	554	1068.311929702758789
612226.04	6073985.08	110	556	1067.647337913513184
.
.
.
626557.62	6085589.72	558	1142	825.678706169128418
626607.60	6085591.12	558	1144	823.747575283050537
626657.58	6085592.52	558	1146	821.8720555305481
626707.56	6085593.91	558	1148	820.536375045776367
```

We made a small library called `gio` (very much a work in progress!) to read files like this. It produces an `xarray.Dataset`, which is a collection of NumPy-array-like things that have Pandas-like indexing wrapped around them. Sounds weird, but they are very useful!

In [ ]:
import gio

fname = spot.fetch('F3_Demo_0_FS4.dat')
data = gio.read_odt(fname)['twt']  # Only one DataArray in this Dataset.
data

One nice thing about DataArrays is that they can plot themselves:

The colorbar orientation is a problem though: I'd prefer it to be the other way up.

Another nice thing is that they can give us their NumPy data; we'll get this and carry on in pure NumPy.

In [ ]:
hor = data.to_numpy()

hor

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">

<h3>EXERCISE</h3>

Plot this dataset using `imshow()`, adding a colorbar. 
</div>

In [ ]:
### YOUR CODE HERE



There's a 'filled contour' option too, `contourf`. Notice that it is plotted with the rows in a different order.

We can also use `plt.contour()`

It's often nice to combine these:

## Interactive plots

The `ipywidgets` package gives us an easy way to make interactive plots. 

About the simplest thing you can do is like this:

In [ ]:
from ipywidgets import interact

@interact(a=(0, 10, 1), b=(0, 100, 10))
def main(a, b):
    """Do the things!"""
    print(a + b)
    return

### EXERCISE

Adapt the code above to make a slider for a plot of the seismic, eg `imshow` a timeslice and make the slider change the timeslice number.

## Adding decoration

So far we've kept most of our calls to matplotlib to one line or so.

Things can get much, much more complicated... The good news is that plots are usually built up, bit by bit. So you start with the one-liner, then gradually add things:

In [ ]:
inline = 100

fig, ax = plt.subplots()
im = ax.imshow(vol[inline, :, :].T, cmap="gray", vmin=-M, vmax=M)
ax.plot(hor[inline, :]/4, 'limegreen', lw=2)  # THIS DOES NOT FIT :(
ax.set_title(f"F3 survey, inline {inline}")
fig.colorbar(im, shrink=0.67)

In [ ]:
# Put everything in seconds.
inl, xl, ts = vol.shape
extent = [0, xl, ts*0.004, 0]  # left, right, bottom, top

fig, ax = plt.subplots()
im = ax.imshow(vol[inline, :, :].T, cmap="gray", vmin=-M, vmax=M, extent=extent, aspect='auto')
ax.plot(hor[inline, :]/1000, 'limegreen', lw=2)
ax.set_title(f"Penobscot, inline {inline}")
ax.set_xlabel("Crossline")
ax.set_ylabel("Time [s]")
fig.colorbar(im, shrink=0.67)

In [ ]:
import matplotlib.patches as patches
import requests
from PIL import Image, ImageOps
import io

zoom = (230, 290, 0.75, 1.4)

fig, axs = plt.subplots(ncols=2, figsize=(15, 6), gridspec_kw={'width_ratios': [2, 1]})

# Plot the main subplot.
ax = axs[0]
im = ax.imshow(vol[inline, :, :].T,
               extent=extent,
               cmap="gray_r",
               vmin=-M, vmax=M,
               aspect='auto',
              )
cb = fig.colorbar(im, ax=ax)
ax.plot(hor[inline, :]/1000, 'limegreen', lw=2)
zl, zr, zt, zb = zoom
rect = patches.Rectangle((zl, zt), zr-zl, zb-zt, lw=1, ec='b', fc='none')
ax.add_patch(rect)
ax.set_title(f"Penobscot, inline {inline}")
ax.set_xlabel("Crossline")
ax.set_ylabel("Time [s]")
ax.text(230, 0.72, "zoom", color='b', fontsize=12)

# Add the polarity cartoon.
r = requests.get("https://app.scienxlab.org/polarity.png?cmap=gray_r")
img = Image.open(io.BytesIO(r.content))
img = ImageOps.expand(img, border=5, fill=(0, 0, 0))
img.thumbnail((100, 100))
fig.figimage(img, 80, 60)

# Plot the zoomed area.
ax = axs[1]
im = ax.imshow(vol[inline].T,
               extent=extent,
               vmin=-M, vmax=M,
               cmap='RdBu',
               aspect='auto',
              )
ax.plot(hor[inline, :]/1000, 'limegreen', lw=2)
for spine in ax.spines.values():
    spine.set_edgecolor('blue')
    spine.set_linewidth(2)
fig.colorbar(im, ax=ax)
ax.set_xlim(zl, zr)
ax.set_ylim(zb, zt)
ax.set_title('Zoomed area', color='b')
ax.set_xlabel("Crossline")

plt.savefig("../data/my_figure.png", dpi=300)
plt.savefig("../data/my_figure.svg")

plt.show()

# How complicated do you want to get?

It turns out you can do almost anything in `matplotlib`. This is a `matplotlib` figure:

In [ ]:
from IPython.display import Image
Image('../images/t1.png')

The key method you need to make a tiled plot like this is [`gridspec`](https://matplotlib.org/users/gridspec.html). You will also need a lot of patience.

## Getting help

- Gallery: https://matplotlib.org/3.1.1/gallery/index.html
- Gallery: https://python-graph-gallery.com/matplotlib/
- Cheatsheets: https://github.com/matplotlib/cheatsheets
- Book: https://github.com/rougier/scientific-visualization-book